In [4]:
import threading
from pathlib import Path

import lib_ipmu_daq_api as api
import lib_ipmu_daq_config as config
import lib_ipmu_daq_generator as gen
import lib_ipmu_daq_process as post
import lib_ipmu_daq_graph as graph

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
preset = config._loadConfig(Path("./_config_preset.toml"))
run = config._loadConfig(Path("./_config_run.toml"))
app_config = config.initParams(preset, run, debug=True)


[Io]
  sample_rate         : 100000
  gen_chunk_sec       : 0.100000
  proc_interval       : 0.125000
  queue_depth         : 40
  quad_depth          : 40

[Gui]
  display_sec         : 3600
  plot_sec            : 0.150000
  gui_interval_ms     : 50
  pruning             : 100

[Logging]
  log_chunk           : 1024
  log_data_num        : 4

[Encoder Postproc]
  quadpulse_width     : 0.000250
  threshold           : 2.500000

[Debug Encoder]
  input_velocity      : 0.100000
  pulse_height        : 5.000000
  pulse_duty          : 0.500000
  pulse_phase_A       : 0.000000

[Debug Power]
  amplitude           : 0.300000
  phase               : 0.000000

[Driver]
  t_DC                : 10
  pps                 : 0
  step                : 100
  rst                 : 1
  target_freq         : 1

[Dependent]
  chunk_sec           : 0.100000
  n_samples_gen       : 10000
  samples_proc        : 12500
  history             : 150
  count_history       : 80
  velo_history        : 80
  pow_

In [ ]:
try:
    app = api.DAQApp(config=app_config, DEBUG=True)
    app.setup() # initialize widgets
    app.initStorer(runs_dir=config.RUNS_DIR) # initialize HDF5 storage
    app.initLogger()  # Initialize logger in the main thread to avoid issues

    # generater for debug
    generator = gen.Generator(config=app_config, buf_q=app.buf_q, stop_event=app.stop_event)

    # processor
    processor = post.Processor(config=app_config, buf_q=app.buf_q, quad_q=app.quad_q, stop_event=app.stop_event, h5f=app.h5f, dset=app.dset, debug=app.DEBUG, logger=app.logger)

    # Graph
    gui = graph.DAQGUI(config=app_config, quad_q=app.quad_q, stop_event=app.stop_event)

    # 5. Create the thread objects individually
    #logger_thread = threading.Thread(target=app.initLogger, daemon=True)
    generator_thread = threading.Thread(target=generator.run, daemon=True)
    processor_thread = threading.Thread(target=processor.run, daemon=True)

    # 6. Start the threads (now controlled from the top level)
    #logger_thread.start()
    generator_thread.start()
    processor_thread.start()

    gui.run()

except KeyboardInterrupt:
    print("KeyboardInterrupt received. Shutting down...")
    app.shutdown()
finally:
    app.shutdown()
    #logger_thread.join(timeout=5)
    generator_thread.join(timeout=5)
    processor_thread.join(timeout=5)



Shared resources (Queues, Stop Event) are set up.
DEBUG mode is ON.
HDF5 dataset created at: C:\Users\USER\Desktop\ipmu_DAQ\src\..\runs\251006144504.h5
EPOCH = 213581.520846, wall = 453.05 ms, jitter = 453.05 ms  delta c=+25, v= 0.098, v_ref= 0.000, time_p = 0.062495, P_tot = 0.415746
EPOCH = 213581.800348, wall = 279.50 ms, jitter = 279.50 ms  delta c=+26, v= 0.102, v_ref= 0.000, time_p = 0.187495, P_tot = 0.415777
EPOCH = 213582.730052, wall = 929.70 ms, jitter = 929.70 ms  delta c=+25, v= 0.098, v_ref= 0.000, time_p = 0.312495, P_tot = 0.415746
EPOCH = 213583.118662, wall = 388.61 ms, jitter = 388.61 ms  delta c=+26, v= 0.102, v_ref= 0.000, time_p = 0.437495, P_tot = 0.415812
EPOCH = 213583.249628, wall = 130.97 ms, jitter = 130.97 ms  delta c=+25, v= 0.098, v_ref= 0.000, time_p = 0.562495, P_tot = 0.415770
EPOCH = 213583.591822, wall = 342.19 ms, jitter = 342.19 ms  delta c=+26, v= 0.102, v_ref= 0.000, time_p = 0.687495, P_tot = 0.415765
EPOCH = 213583.712737, wall = 120.91 ms, jit